
# CS301 Midterm Project — EDA & Visualization (YouTube Data)

**Student:** Elvis B. Otieno  
**Course:** CS301 (Data Science)  
**Project:** Exploratory Data Analysis & Visualization  
**Dataset:** `olaolatunde/data-for-youtube` (Kaggle)

> In this notebook I walk through my process: how I picked the dataset, how I cleaned it,
> what I decided to keep/drop, what I visualized, and what I learned. I kept the tone practical
> and focused on telling a clear data story.


In [ ]:
pip install kagglehub pandas matplotlib numpy


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [ ]:

# 0) Setup
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# I prefer readable numbers
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")



## 1) Dataset Description (what it is and what I want to learn)

**Source.** I used the Kaggle dataset **Data for YouTube** (`olaolatunde/data-for-youtube`).
It contains video-level metrics like titles, views, likes, and comments. This is perfect for
a lightweight EDA because the columns are understandable and the questions are intuitive.

**My questions.**
1. Which factors seem most related to video popularity (I use *views* and *likes* as the proxies)?  
2. Do engagement ratios (likes per view, comments per view) look different across categories or across short/medium/long videos?

**Why this matters.** If I were a creator, I’d want to know which knobs are associated with more engagement,
and whether certain categories do better per view than others.



## 2) Getting the Data (Kaggle + fallback)

I first try to download via `kagglehub`. If that doesn’t work on your machine, you can
manually download the CSV from Kaggle and set `manual_csv_path` below.


In [ ]:

# 2.1) Try kagglehub, then search for CSVs in the cache
csv_candidates = []
dataset_path = None

try:
    import kagglehub
    dataset_path = kagglehub.dataset_download("olaolatunde/data-for-youtube")
    print("Kaggle dataset cached at:", dataset_path)
    for root, _, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith(".csv"):
                csv_candidates.append(os.path.join(root, f))
except Exception as e:
    print("Kaggle download not available here:", e)

csv_candidates


In [ ]:

# 2.2) Fallback: set this to your local CSV path if Kaggle download is unavailable
manual_csv_path = "youtube_data.csv"  # update if you saved a local copy

def choose_csv(cands):
    if not cands:
        return None
    yt_like = [p for p in cands if "youtube" in os.path.basename(p).lower()]
    pool = yt_like if yt_like else cands
    # pick the largest file as a simple heuristic
    return max(pool, key=lambda p: os.path.getsize(p))

csv_path = choose_csv(csv_candidates)

if (csv_path is None) and os.path.exists(manual_csv_path):
    csv_path = manual_csv_path

print("Using CSV:", csv_path)
if csv_path is None:
    raise FileNotFoundError("No CSV found yet. Set manual_csv_path to a valid local file.")



## 3) First look at the raw data

Here I load the CSV, check shape and types, and preview a few rows. This helps me decide
what columns matter and where the messy parts are (missingness, text vs numeric, etc.).


In [ ]:

import os, csv
from itertools import islice

print("Reading:", csv_path, "| size:", os.path.getsize(csv_path), "bytes")

def robust_read_csv(path):
    # Try to sniff delimiter first
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        sample = "".join(list(islice(f, 200)))
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",",";","|","\t"])
        sep = dialect.delimiter
    except Exception:
        sep = None  # let pandas infer

    # Most forgiving parse first
    try:
        df0 = pd.read_csv(
            path,
            sep=sep,                # None -> infer; else use sniffed
            engine="python",        # more tolerant than C engine
            encoding="utf-8",
            encoding_errors="replace",
            on_bad_lines="skip",    # skip malformed records
        )
        return df0
    except Exception as e1:
        print("First pass failed:", e1)

    # Try common alt encodings / delimiters
    for enc in ["utf-8-sig", "latin-1"]:
        for alt_sep in [sep, ",", ";", "\t", "|", None]:
            try:
                df0 = pd.read_csv(
                    path,
                    sep=alt_sep,
                    engine="python",
                    encoding=enc,
                    encoding_errors="replace",
                    on_bad_lines="skip",
                )
                print(f"Loaded with encoding={enc}, sep={alt_sep!r}")
                return df0
            except Exception as e2:
                last_err = e2
    raise last_err

df_raw = robust_read_csv(csv_path)
df = df_raw.copy()
print("Loaded shape:", df.shape)
df.head()



# User
yes


## 4) Cleaning & preparation (what I changed and why)

What I chose to do:
- **Dedup** rows so each video appears once.
- **Standardize** column names to snake_case (less typing errors later).
- **Coerce types**: make obvious numeric columns numeric and parse dates where it makes sense.
- **Missing values**: I drop rows missing *key* numeric metrics (views/likes/comments) since imputing
  those would make the ratios unreliable. Categorical nulls I fill with `"Unknown"` to keep rows.
- **Feature engineering**: `like_rate = likes / views`, `comment_rate = comments / views`.
  If there’s a duration column, I bucket it into Short/Medium/Long (terciles) to compare groups.


In [ ]:

# 4.1) Deduplicate
before = len(df)
df = df.drop_duplicates()
print(f"Deduplicated: {before - len(df)} rows removed (now {len(df)})")

# 4.2) Standardize column names
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# 4.3) Identify columns by name patterns (flexible across dataset versions)
num_hints  = ["view", "like", "comment", "duration", "dislike", "share"]
cat_hints  = ["category", "channel", "title"]
date_hints = ["date", "published"]

likely_numeric = [c for c in df.columns if any(k in c for k in num_hints)]
likely_cats    = [c for c in df.columns if any(k in c for k in cat_hints)]
likely_dates   = [c for c in df.columns if any(k in c for k in date_hints)]

print("Likely numeric:", likely_numeric)
print("Likely categorical:", likely_cats)
print("Likely datetime:", likely_dates)

# 4.4) Convert types
for c in likely_dates:
    try:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    except Exception:
        pass

for c in likely_numeric:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 4.5) Missingness report
missing_report = df.isna().mean().sort_values(ascending=False)
missing_report.head(12)


In [ ]:

# 4.6) Handle missing values
central_numeric = [c for c in ["views","likes","comments"] if c in df.columns]

for c in central_numeric:
    before = len(df)
    df = df[~df[c].isna()]
    print(f"Dropped {before - len(df)} rows due to missing {c}. (Now {len(df)})")

for c in df.select_dtypes(include="object").columns:
    if df[c].isna().any():
        df[c] = df[c].fillna("Unknown")

# 4.7) Light outlier capping so charts aren't dominated by a few massive points
for c in central_numeric:
    if c in df.columns:
        cap = df[c].quantile(0.999)
        df[c] = np.minimum(df[c], cap)

print("Cleaning complete.")


In [ ]:

# 4.8) Feature engineering
if "views" in df.columns:
    if "likes" in df.columns:
        df["like_rate"] = df["likes"] / df["views"].replace(0, np.nan)
    if "comments" in df.columns:
        df["comment_rate"] = df["comments"] / df["views"].replace(0, np.nan)

# duration buckets if any duration-like column is present
dur_col = None
for c in df.columns:
    if "duration" in c:
        dur_col = c
        break

if dur_col is not None:
    if df[dur_col].dtype == object:
        df[dur_col] = pd.to_numeric(df[dur_col], errors="coerce")
    if df[dur_col].notna().sum() > 0:
        q = df[dur_col].quantile([0.33, 0.66]).values
        def bucket(x):
            if pd.isna(x): return "Unknown"
            if x <= q[0]: return "Short"
            if x <= q[1]: return "Medium"
            return "Long"
        df["duration_bucket"] = df[dur_col].apply(bucket)

df.head(8)



## 5) EDA — What does the data say at a glance?

I start with simple descriptives and a correlation matrix. My expectation is that views/likes/comments
all move together (exposure → engagement), but engagement *rates* might separate by category/duration.


In [ ]:

desc = df.describe(include="all")




In [ ]:

num_df = df.select_dtypes(include=[np.number])
corr = num_df.corr(numeric_only=True)
corr



## 6) Visualizations (plain matplotlib, one chart per figure)

> Per my class guidelines, I used **matplotlib only**, **no custom colors**, and **one plot per figure**.



### 6.1) Histogram — Views
I expected a long right tail (viral videos). The histogram helps confirm that most videos are modest, a few are huge.


In [ ]:

if "views" in df.columns:
    plt.figure()
    df["views"].dropna().plot(kind="hist", bins=50)
    plt.title("Distribution of Views")
    plt.xlabel("Views")
    plt.ylabel("Frequency")
    plt.show()
else:
    print("Skipping: 'views' column not found.")



### 6.2) Scatter — Likes vs Views (log scales)
If views and likes scale together, a diagonal cloud should appear on a log–log scatter.


In [ ]:

if all(c in df.columns for c in ["views","likes"]):
    plt.figure()
    plt.scatter(df["views"], df["likes"], s=9, alpha=0.5)
    plt.xscale("log")
    plt.yscale("log")
    plt.title("Likes vs Views (log–log)")
    plt.xlabel("Views")
    plt.ylabel("Likes")
    plt.show()
else:
    print("Skipping: need both 'views' and 'likes'.")



### 6.3) Boxplot — Like Rate by Category
I wanted to see if some categories punch above their weight (higher likes per view). If the dataset has a category column, this shows it.


In [ ]:

cat_col = None
for c in df.select_dtypes(include=["object"]).columns:
    if "category" in c:
        cat_col = c
        break

if cat_col is not None and "like_rate" in df.columns:
    groups, labels = [], []
    for k, g in df[[cat_col, "like_rate"]].dropna().groupby(cat_col):
        vals = g["like_rate"].values
        if len(vals) > 0:
            groups.append(vals)
            labels.append(str(k)[:18])

    if groups:
        plt.figure()
        plt.boxplot(groups, showfliers=False)
        plt.title("Like Rate by Category")
        plt.xlabel("Category")
        plt.ylabel("Likes / Views")
        plt.xticks(range(1, len(labels)+1), labels, rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
    else:
        print("No non-empty groups for category vs like_rate.")
else:
    print("Skipping: category and/or like_rate not available.")



### 6.4) Heatmap — Numeric Correlations
This is a quick overview of which numeric features move together.


In [ ]:

if 'corr' in globals() and corr.size > 0:
    plt.figure()
    im = plt.imshow(corr.values, aspect="auto")
    plt.title("Correlation Heatmap (Numeric)")
    plt.xlabel("Features")
    plt.ylabel("Features")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
    plt.yticks(range(len(corr.index)), corr.index)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping heatmap: no numeric correlation matrix.")



## 7) What I learned (and what I'd try next)

**Takeaways.**
- **Exposure drives engagement:** Views and likes typically rise together (strong positive correlation),
  which matches my expectations for platform dynamics.
- **Efficiency differs by content:** Like-per-view (and often comment-per-view) varies by category and
  sometimes by duration bucket. Some topics attract more *per-view* enthusiasm than others.
- **Heavy right tail:** A small fraction of videos dominate total engagement, so I capped extreme outliers
  slightly to keep plots readable.

**If I had more time**, I’d add a simple regression with log(views) as the target to see which features
hold up after controlling for others, and I’d check robustness with different outlier rules.



## Machine Learning: Predicting High-View YouTube Videos

In this section, we extend our exploratory data analysis by building machine learning models
to predict whether a video will achieve **high view counts** based on its features.
We will:

1. Define a binary target variable `high_views` based on the median number of views.
2. Engineer a few additional features (engagement ratios).
3. Split the data into training and test sets using a fixed random state.
4. Train and evaluate two models on the same data:
   - Logistic Regression (linear baseline model)
   - Random Forest (non-linear tree-based model)
5. Compare their performance using accuracy, precision, recall, F1-score, and confusion matrices.


In [ ]:

import numpy as np
import pandas as pd

# We assume the main cleaned DataFrame is named df and already exists from the EDA section.
# If your main DataFrame has a different name, update it here accordingly.

# Drop rows with missing views, likes, or comments (these are essential for our prediction task)
df_ml = df.dropna(subset=["views", "likes", "comments"]).copy()

# Define a binary target: 1 if views >= median, else 0
view_threshold = df_ml["views"].median()
df_ml["high_views"] = (df_ml["views"] >= view_threshold).astype(int)

print("View threshold (median):", view_threshold)
print("\nClass distribution (high_views):")
print(df_ml["high_views"].value_counts())
print("\nClass distribution (proportion):")
print(df_ml["high_views"].value_counts(normalize=True))



### Feature Engineering and Selection

We create additional features that capture engagement intensity and then select a set of
input features for our models.

- **likes_per_view**: likes divided by views (how many likes per view)
- **comments_per_view**: comments divided by views (how many comments per view)

We also one-hot encode the `category` column (if present) so that models can use category
information as numeric inputs.


In [ ]:

# Basic engagement features
df_ml["likes_per_view"] = df_ml["likes"] / (df_ml["views"] + 1)
df_ml["comments_per_view"] = df_ml["comments"] / (df_ml["views"] + 1)

# One-hot encode category if available
cat_cols = []
if "category" in df_ml.columns:
    cat_cols = ["category"]
    df_ml = pd.get_dummies(df_ml, columns=cat_cols, drop_first=True)

# Core numeric features
feature_cols = [
    "likes",
    "comments",
    "likes_per_view",
    "comments_per_view"
]

# Add any one-hot category columns automatically
feature_cols += [c for c in df_ml.columns if c.startswith("category_")]

X = df_ml[feature_cols]
y = df_ml["high_views"]

print("Number of samples:", X.shape[0])
print("Number of features:", X.shape[1])
print("Feature columns:")
print(feature_cols)



### Train–Test Split

We split the data into training and test sets using an 80/20 split.
To make the comparison fair between models, we use the **same split** and a fixed
`random_state` value. We also stratify on the target variable to maintain the same
class distribution in both sets.


In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])



### Model 1: Logistic Regression

Logistic Regression is a linear classification model that is:

- Fast and efficient.
- Reasonably interpretable: each coefficient shows how a feature affects the log-odds
  of a video being in the **high_views** class.
- A good **baseline** model to compare against more complex models.


In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

log_reg_clf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000))
])

log_reg_clf.fit(X_train, y_train)
y_pred_lr = log_reg_clf.predict(X_test)

print("Logistic Regression — Classification Report")
print(classification_report(y_test, y_pred_lr))

print("Confusion Matrix (Logistic Regression)")
print(confusion_matrix(y_test, y_pred_lr))



### Model 2: Random Forest

Random Forest is an ensemble of decision trees and is able to:

- Capture non-linear relationships and interactions between features.
- Handle a mix of feature types (numeric and one-hot encoded).
- Provide feature importance scores indicating which variables are most useful.

It is typically less interpretable than Logistic Regression, but often achieves
stronger predictive performance.


In [ ]:

from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)

print("Random Forest — Classification Report")
print(classification_report(y_test, y_pred_rf))

print("Confusion Matrix (Random Forest)")
print(confusion_matrix(y_test, y_pred_rf))



### Random Forest Feature Importance

We can also inspect the feature importances from the Random Forest model to see which
variables contributed most to the predictions.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

importances = rf_clf.feature_importances_
idx = np.argsort(importances)[::-1]

plt.figure()
plt.bar(range(len(feature_cols)), importances[idx])
plt.xticks(range(len(feature_cols)), np.array(feature_cols)[idx], rotation=90)
plt.ylabel("Importance")
plt.title("Random Forest Feature Importances")
plt.tight_layout()
plt.show()



### Model Performance Comparison

We now compare Logistic Regression and Random Forest side-by-side using accuracy,
precision, recall, and F1-score. This allows us to see which model performs better
overall and how they trade off between different metrics.


In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def summarize_model(name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }

summary = []
summary.append(summarize_model("Logistic Regression", y_test, y_pred_lr))
summary.append(summarize_model("Random Forest", y_test, y_pred_rf))

summary_df = pd.DataFrame(summary)
summary_df



### Discussion and Conclusions

- **Which model performed better?**  
  Use the comparison table above to decide which model has higher F1-score and/or
  accuracy. In many cases, Random Forest will outperform Logistic Regression when
  relationships are non-linear, but this depends on the data.

- **Strengths of Logistic Regression in this project:**  
  - Simple and fast to train.  
  - Provides interpretable coefficients (direction of effect for each feature).  
  - Good baseline for comparison.

- **Strengths of Random Forest in this project:**  
  - Captures more complex relationships between engagement features and high views.  
  - Provides feature importance scores, highlighting which variables matter most.  

- **Limitations and future work:**  
  - We only used a small set of features (likes, comments, and basic ratios).  
  - Text features (title/description), thumbnail quality, or upload time could
    provide additional signal.  
  - Hyperparameter tuning (e.g., grid search) could further improve performance.

Overall, this modeling exercise complements the EDA by showing that engagement metrics
(likes, comments, and related ratios) can be used to build predictive models for
video popularity, and that more flexible models like Random Forest may capture
patterns that linear models miss.
